## True Heroes of the Great Web. Genesis and ETH2 Stakers

In [1]:
from config import INITIAL_TS, SNAPSHOT_TS, \
    GENESIS_AND_ETH2_DATASET_NAME, GENESIS_AND_ETH2_DISTRIBUTION_TABLE_NAME
from src.utils_bigquery import drop_table, create_table

DROP_TABLES = True
CREATE_TABLES = True

### Distribution Rule
<table style="text-align: center">
    <thead style="text-align: center">
        <tr>
            <th rowspan=2></th>
            <th colspan=3>Grade</th>
        </tr>
        <tr>
            <th>1</th>
            <th>2</th>
            <th>3</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align: left"> Genesis </td>
            <td> - </td>
            <td> - </td>
            <td> only here </td>
        </tr>
        <tr>
            <td style="text-align: left"> ETH 2.0 Stakers </td>
            <td> - </td>
            <td> - </td>
            <td> only here </td>
        </tr>
    </tbody>
</table>

### Genesis and ETH 2.0 Stakers

In [2]:
ETH2_STAKE_CONTRACT_ADDRESS = '0x00000000219ab540356cbb839cbe05303d7705fa'
ETH2_STAKE_START_TS = '2020-11-03 00:00:00'

query_1 = f'''
    WITH traces AS (
       SELECT
          transaction_hash,
          value
       FROM `bigquery-public-data.crypto_ethereum.traces`
       WHERE block_timestamp >= '{ETH2_STAKE_START_TS}'
         AND value >= 1e18
         AND status = 1
         AND to_address = '{ETH2_STAKE_CONTRACT_ADDRESS}'
    )
    SELECT
       'ETH 2.0 Staker' as segment,
       address,
       3 as grade,
       sum(value)/1e18 as sum_value
    FROM (
       SELECT
          traces.transaction_hash as transaction_hash,
          traces.value as value,
          transactions.from_address as address
       FROM traces
       INNER JOIN `bigquery-public-data.crypto_ethereum.transactions` as transactions
       ON traces.transaction_hash = transactions.hash
       WHERE transactions.block_timestamp >= '{ETH2_STAKE_START_TS}'
         AND transactions.block_timestamp >= '{INITIAL_TS}'
         AND transactions.block_timestamp <= '{SNAPSHOT_TS}')
    GROUP BY address

    UNION ALL

    SELECT
        'Genesis' as segment,
        to_address as address,
        3 as grade,
        sum(value)/1e18 as sum_value
    FROM `bigquery-public-data.crypto_ethereum.traces`
    WHERE block_timestamp < '2015-08-01 00:00:00'
      AND block_number=0
      AND trace_type='genesis'
    GROUP BY address
'''

if DROP_TABLES:
    drop_table(table_name=GENESIS_AND_ETH2_DISTRIBUTION_TABLE_NAME,
               dataset_name=GENESIS_AND_ETH2_DATASET_NAME)

if CREATE_TABLES:
    create_table(query=query_1,
                 table_name=GENESIS_AND_ETH2_DISTRIBUTION_TABLE_NAME,
                 dataset_name=GENESIS_AND_ETH2_DATASET_NAME)

Table genesis_and_eth2_stakers:genesis_and_eth2_stakers_distribution has been deleted.
Table genesis_and_eth2_stakers:genesis_and_eth2_stakers_distribution has been created and filled 18679 rows.
